<a href="https://colab.research.google.com/github/uchihamadara37/ML-capstone-bangkit/blob/main/andre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


Setup Library
**Preprocessing**

In [2]:
# prompt: install kaggle library

!pip install kaggle


In [3]:
!kaggle datasets download -d wangziang/waste-pictures -p data --unzip

Dataset URL: https://www.kaggle.com/datasets/wangziang/waste-pictures
License(s): unknown
 99% 2.03G/2.05G [00:29<00:00, 79.5MB/s]
100% 2.05G/2.05G [00:29<00:00, 74.9MB/s]


## **Exploratory Analysis**
First, I need to prepare all libraries needed for analysis.

In [12]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path

In [8]:
for dirname, _, filenames in os.walk('data'):
    print(dirname)
    for filename in filenames:
        print(os.path.join(dirname, filename))

print("ok")

Streaming output truncated to the last 5000 lines.
data/train/plasticene/plasticene (477).jpg
data/train/plasticene/plasticene (700).jpg
data/train/plasticene/plasticene (40).jpg
data/train/plasticene/plasticene (368).jpg
data/train/plasticene/plasticene (503).jpg
data/train/plasticene/plasticene (661).jpg
data/train/plasticene/plasticene (606).jpg
data/train/plasticene/plasticene (486).jpg
data/train/plasticene/plasticene (484).jpg
data/train/plasticene/plasticene (39).jpg
data/train/plasticene/plasticene (287).jpg
data/train/plasticene/plasticene (203).jpg
data/train/plasticene/plasticene (314).jpg
data/train/plasticene/plasticene (229).jpg
data/train/plasticene/plasticene (619).jpg
data/train/plasticene/plasticene (301).jpg
data/train/plasticene/plasticene (210).jpg
data/train/plasticene/plasticene (64).jpg
data/train/plasticene/plasticene (575).jpg
data/train/plasticene/plasticene (16).jpg
data/train/plasticene/plasticene (399).jpg
data/train/plasticene/plasticene (452).jpg
data/tr

In [14]:
root_dir = Path('./data')
train_dir = 'data/train'
test_dir = 'data/test'

In [21]:

for entry in os.listdir(root_dir):
    full_path = os.path.join(root_dir, entry)
    if os.path.isdir(full_path):
        print(f"|---->: {full_path}")
        for entry2 in os.listdir(full_path):
            full_path2 = os.path.join(full_path, entry2)
            if os.path.isdir(full_path2):
                print(f"    |---->: {full_path2}")


|---->: data/test
    |---->: data/test/chopsticks
    |---->: data/test/plasticbottle
    |---->: data/test/tabletcapsule
    |---->: data/test/cans
    |---->: data/test/glassbottle
    |---->: data/test/thermometer
    |---->: data/test/toothpick
    |---->: data/test/napkin
    |---->: data/test/watermelonrind
    |---->: data/test/XLight
    |---->: data/test/bowlsanddishes
    |---->: data/test/leftovers
    |---->: data/test/penholder
    |---->: data/test/facialmask
    |---->: data/test/leaflet
    |---->: data/test/medicinebottle
    |---->: data/test/pesticidebottle
    |---->: data/test/carton
    |---->: data/test/battery
    |---->: data/test/bandaid
    |---->: data/test/plasticbag
    |---->: data/test/nut
    |---->: data/test/toothpastetube
    |---->: data/test/rag
    |---->: data/test/plasticene
    |---->: data/test/nailpolishbottle
    |---->: data/test/milkbox
    |---->: data/test/traditionalChinesemedicine
    |---->: data/test/cigarettebutt
    |---->: data/t